### Import Libraries

In [1]:
# add path to custom python code for accessing data lake and working with dataframes
import sys
sys.path.append('/Users/markbills/Library/CloudStorage/OneDrive-Transformativ,LLC/Clients/Ovation Holdings/src')

# Azure Data Lake libraries
import azure_data_lake_interface as adl

# Helper function libraries
import helper_functions as hf

# Data analysis libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import weekly_margin_report as wmr

# display support
from IPython.display import display, Markdown

### Load Data

In [3]:
# attach to the data lake
config = hf.load_config("config/datalake_config.json", flush_cache=True)
service_client = adl.get_azure_service_client(config["blob_url"])
file_system_client = adl.get_azure_file_system_client(service_client, "consolidated")

# get data lake data
trans_type = "Estimate"
filename = f"transaction/{trans_type}ItemLineItems_enhanced.parquet"
line_items = adl.get_parquet_file_from_data_lake(file_system_client, "enhanced/netsuite", filename)

### Explore Data

In [ ]:
subset = line_items[['item_type', 'manufacturer', 'item_name', 'description', 'quantity',
                                'total_amount', 'quote_po_rate', '12_month_max_unit_cost']].copy()

In [19]:
subset.dropna(inplace=True)
subset.avg_quoted_cost = round(subset.avg_quoted_cost, 2)
subset = subset[subset['monthly_inventory_sales'] > 0]
subset['cost_difference'] = subset['avg_quoted_cost'] - subset['12_month_max_unit_cost']
subset

item_type        manufacturer  \
month   sku                                           
2022-01 100      Inventory Item              Kunkle   
        1005562  Inventory Item  Automation Service   
        10089    Inventory Item              Farris   
        1012     Inventory Item              Kunkle   
        10186    Inventory Item              Farris   
...                         ...                 ...   
2025-04 9328     Inventory Item        Consolidated   
        9381     Inventory Item        Consolidated   
        9610     Inventory Item        Consolidated   
        9632     Inventory Item              Farris   
        9886     Inventory Item        Consolidated   

                                         item_name  \
month   sku                                          
2022-01 100          912BDCM01  .5 x .75 KUNKLE SV   
        1005562                            1005562   
        10089    GK16 Farris Gasket Set (Standard)   
        1012            20C02 .5x.5 KUNKLE SRV MxF   
        10186                      GK27 Farris 26R   
...                                            ...   
2025-04 9328                                39PTKL   
        9381                           4441803 -PH   
        9610                   SP540-JKIT SWAGELOK   
        9632               4KG26A22 (formerly GK9)   
        9886                           PSGK-38K014   

                                                       description  \
month   sku                                                          
2022-01 100                      912BDCM01\r\n.5" x .75" KUNKLE SV   
        1005562  Fisher RPACKX00022 Packing Kit  (Non Live- Loa...   
        10089                                      GK16 Gasket Kit   
        1012     20-C02 \r\n.5" KUNKLE SV\r\n(.5" MNPT x .5" FNPT)   
        10186                                      GK27 Gasket Kit   
...                                                            ...   
2025-04 9328                               39PTKL Fitting/Tube Kit   
        9381                                     4441803-PH NOZZLE   
        9610                  SP540-JKIT (SS-8F-K4-60,  SS-8TF-K2)   
        9632                                   4KG26A22 Gasket Kit   
        9886                                 PSGK-38K014 Pilot Kit   

                 monthly_inventory_qty  monthly_inventory_sales  \
month   sku                                                       
2022-01 100                        3.0                   921.26   
        1005562                    1.0                    32.00   
        10089                      2.0                   453.00   
        1012                       1.0                   144.96   
        10186                      3.0                   769.65   
...                                ...                      ...   
2025-04 9328                       1.0                   466.00   
        9381                       1.0                  4015.55   
        9610                       1.0                   135.52   
        9632                       1.0                   237.00   
        9886                       1.0                  2247.00   

                 avg_quoted_cost  12_month_max_unit_cost  cost_difference  
month   sku                                                                
2022-01 100               131.05                  194.40           -63.35  
        1005562            27.00                   27.00             0.00  
        10089             134.00                  147.00           -13.00  
        1012              105.70                  105.70             0.00  
        10186             160.33                  165.00            -4.67  
...                          ...                     ...              ...  
2025-04 9328              207.15                  508.92          -301.77  
        9381             2691.15                 3011.86          -320.71  
        9610               71.91                  205.01    

In [20]:
subset_lines = subset.shape[0]
subset_max_price = subset.query('cost_difference >= 0').shape[0]
print(f"Pct of items where average monthly quoted cost was greater than or equal to 12-month maximum cost: {(subset_max_price/subset_lines)*100:.2f}%")

Pct of items where average monthly quoted cost was greater than or equal to 12-month maximum cost: 61.79%


In [21]:
below_max = subset.query('cost_difference < 0')
below_max = below_max[below_max['manufacturer'] != 'null']
ttm_below_max = below_max.loc[below_max.index.get_level_values('month') >= '2024-04'].copy()
ttm_below_max.rename(columns={'monthly_inventory_qty': 'qty', 'monthly_inventory_sales': 'sales', 'avg_quoted_cost': 'quoted_cost',
                              'cost_difference': 'difference', '12_month_max_unit_cost': 'max_cost'}, inplace=True)
ttm_below_max['pct_diff'] = round((ttm_below_max.difference / ttm_below_max.max_cost) * 100, 2)
ttm_below_max.describe().iloc[1:]

,qty,sales,quoted_cost,max_cost,difference,pct_diff
mean,86.201941,6253.075939,823.979993,1231.750441,-407.770449,-18.820219
std,585.036968,16885.640997,2217.277377,7373.162889,5907.939255,20.046046
min,0.250000,3.400000,0.000000,0.042900,-306819.380000,-100.000000
25%,1.000000,430.000000,53.565000,73.000000,-133.912500,-26.965000
50%,3.000000,1504.440000,220.525000,272.000000,-25.405000,-11.805000
75%,9.000000,5229.000000,740.140000,917.645000,-4.150000,-4.337500
max,25920.000000,529907.580000,95028.380000,401847.760000,-0.001000,-0.000000


In [22]:
ttm_below_max

item_type  manufacturer  \
month   sku                                     
2024-04 100      Inventory Item        Kunkle   
        10066    Inventory Item         Groth   
        10071    Inventory Item  Consolidated   
        10096    Inventory Item  Consolidated   
        1011373  Inventory Item       Protego   
...                         ...           ...   
2025-04 9328     Inventory Item  Consolidated   
        9381     Inventory Item  Consolidated   
        9610     Inventory Item  Consolidated   
        9632     Inventory Item        Farris   
        9886     Inventory Item  Consolidated   

                                                    item_name  \
month   sku                                                     
2024-04 100                     912BDCM01  .5 x .75 KUNKLE SV   
        10066                                    KSPVRVW06TVT   
        10071          Inactivated + 4463501 DISC 1997 01.500   
        10096    Inactivated + 5039101 BOTTOM ADJUSTER 39PV07   
        1011373                                       8550400   
...                                                       ...   
2025-04 9328                                           39PTKL   
        9381                                      4441803 -PH   
        9610                              SP540-JKIT SWAGELOK   
        9632                          4KG26A22 (formerly GK9)   
        9886                                      PSGK-38K014   

                                          description   qty    sales  \
month   sku                                                            
2024-04 100         912BDCM01\r\n.5" x .75" KUNKLE SV  11.0  2625.00   
        10066                          Soft Goods Kit   6.0  4771.05   
        10071                4463501 DISC 1997 01.500   1.0   461.32   
        10096            5039101 BOTTOM ADJUSTER 39PV   1.0   832.00   
        1011373                      8550400 - Gasket   2.0  1315.00   
...                                               ...   ...      ...   
2025-04 9328                  39PTKL Fitting/Tube Kit   1.0   466.00   
        9381                        4441803-PH NOZZLE   1.0  4015.55   
        9610     SP540-JKIT (SS-8F-K4-60,  SS-8TF-K2)   1.0   135.52   
        9632                      4KG26A22 Gasket Kit   1.0   237.00   
        9886                    PSGK-38K014 Pilot Kit   1.0  2247.00   

                 quoted_cost  max_cost  difference  pct_diff  
month   sku                                                   
2024-04 100           153.30    224.10      -70.80    -31.59  
        10066         529.60    568.33      -38.73     -6.81  
        10071         392.11    392.12       -0.01     -0.00  
        10096         374.80    431.02      -56.22    -13.04  
        1011373       375.00    725.00     -350.00    -48.28  
...                      ...       ...         ...       ...  
2025-04 9328          207.15    508.92     -301.77    -59.30  
        9381         2691.15   3011.86     -320.71    -10.65  
        9610           71.91    205.01     -133.10    -64.92  
        9632          154.00    182.00      -28.00    -15.38  
        9886         1152.02   2690.00    -1537.98    -57.17  

[6748 rows x 10 columns]

In [27]:
print(f"Total TTM Sales with Below Max Quoted Costs: ${ttm_below_max.sales.sum():,.2f}")

Total TTM Sales with Below Max Quoted Costs: $42,195,756.44


In [33]:
item_type = "Inventory Item"
manufacturer = "Yokogawa"
mfr_subset = ttm_below_max[(ttm_below_max.manufacturer == manufacturer) & (ttm_below_max.item_type == item_type)]
mfr_subset.describe().iloc[1:]

,qty,sales,quoted_cost,max_cost,difference,pct_diff
mean,5.259036,6568.225657,1201.694337,1557.812289,-356.117952,-20.661175
std,6.519254,9915.394013,1256.577227,1545.741252,491.652125,14.428955
min,1.000000,20.000000,2.850000,3.000000,-3389.950000,-61.690000
25%,1.000000,990.837500,394.105000,488.000000,-504.727500,-30.160000
50%,3.000000,2724.250000,914.975000,1155.000000,-161.725000,-17.500000
75%,6.000000,7600.312500,1468.515000,2136.000000,-49.947500,-9.917500
max,35.000000,72804.740000,7451.000000,10756.750000,-0.090000,-0.040000


In [32]:
mfr_subset

item_type manufacturer  \
month   sku                                    
2024-04 1548369  Inventory Item     Yokogawa   
        1548370  Inventory Item     Yokogawa   
        1548382  Inventory Item     Yokogawa   
        1548497  Inventory Item     Yokogawa   
        1548583  Inventory Item     Yokogawa   
...                         ...          ...   
2025-04 1550443  Inventory Item     Yokogawa   
        1562873  Inventory Item     Yokogawa   
        1570058  Inventory Item     Yokogawa   
        1633953  Inventory Item     Yokogawa   
        1670659  Inventory Item     Yokogawa   

                                              item_name  \
month   sku                                               
2024-04 1548369                             C13SA-SRPBN   
        1548370               C13ST-2GSCT-NN-NNPS1-NNNN   
        1548382               C13ST-5TSA0-S4-NNPS2-NNNN   
        1548497                                 E7042UD   
        1548583              EJA510E-JBS4N-012EL/FU1/D1   
...                                                 ...   
2025-04 1550443                            PH72SN-11-AA   
        1562873                             PH97-07/05M   
        1570058                    AXG4A-CFF2121JE11/MC   
        1633953  FLXA402-A-B-AD-C5-NN-A2-WR-N-N-N-NN/UM   
        1670659            EJA530E-JDS4N-012EN/FU1/A/D1   

                                                       description   qty  \
month   sku                                                                
2024-04 1548369  C13SA-SRPBN\r\n\r\nYokogawa Manifold Gasket, T...   8.0   
        1548370  C13ST-2GSCT-NN-NNPS1-NNNN\r\n\r\nYokogawa Bloc...   6.0   
        1548382  C13ST-5TSA0-S4-NNPS2-NNNN\r\n\r\nYokogawa 5-Va...   5.0   
        1548497  E7042UD \r\n\r\nYokogawa Universal O2 Cell Ass...  29.0   
        1548583  EJA510E-JBS4N-012EL/FU1/D1 - Yokogawa Absolute...   4.0   
...                                                            ...   ...   
2025-04 1550443  PH72SN-11-AA\r\n\r\nYokogawa pH Sensor For PH7...   2.0   
        1562873  PH97-07/05M \r\n\r\nCombination pH Sensor for ...   1.0   
        1570058  Magnetic Flowmeter Remote Transmitter\r\nFM Fl...   2.0   
        1633953  FLXA402-A-B-AD-C5-NN-A2-WR-N-N-N-NN/UM\r\n\r\n...   2.0   
        1670659  Yokogawa DPharp Digital Direct Mount Gauge Pre...   3.0   

                    sales  quoted_cost  max_cost  difference  pct_diff  
month   sku                                                             
2024-04 1548369     56.00         4.62       7.0       -2.38    -34.00  
        1548370    642.00        94.97     112.0      -17.03    -15.21  
        1548382   2125.00       318.75     457.0     -138.25    -30.25  
        1548497  33057.76       956.26    1096.0     -139.74    -12.75  
        1548583   4549.90      1179.71    1507.9     -328.19    -21.76  
...                   ...          ...       ...         ...       ...  
2025-04 1550443    598.00       196.35     238.0      -41.65    -17.50  
        1562873    642.00       469.26     642.0     -172.74    -26.91  
        1570058   3736.00      1401.00    2335.0     -934.00    -40.00  
        1633953   4272.00      1602.00    2136.0     -534.00    -25.00  
        1670659   4658.85       943.36    1722.0     -778.64    -45.22  

[332 rows x 10 columns]

In [31]:
print(f"Total TTM Sales with Below Max Quoted Costs for {manufacturer}: ${mfr_subset.sales.sum():,.2f}")

Total TTM Sales with Below Max Quoted Costs for Yokogawa: $2,180,650.92


In [26]:
ttm_below_max.to_excel('data_tables/ttm_quoted_cost_below_twelve_month_max_cost.xlsx')

In [52]:
above_max = subset.query('cost_difference > 1').copy()
above_max = above_max[above_max['manufacturer'] != 'null']
ttm_above_max = above_max.loc[above_max.index.get_level_values('month') >= '2024-04'].copy()
ttm_above_max.rename(columns={'monthly_inventory_qty': 'qty', 'monthly_inventory_sales': 'sales', 'avg_quoted_cost': 'quoted_cost',
                              'cost_difference': 'difference', '12_month_max_unit_cost': 'max_cost'}, inplace=True)
ttm_above_max['pct_diff'] = round((ttm_above_max.difference / ttm_above_max.max_cost) * 100, 2)
ttm_above_max.describe().iloc[1:]

,qty,sales,quoted_cost,max_cost,difference,pct_diff
mean,12.771327,5933.030877,2010.991090,1691.987387,319.003703,72.844929
std,160.081691,10433.938064,3218.767877,2797.556102,1065.689280,561.375216
min,0.500000,19.000000,5.000000,0.153100,1.040000,0.020000
25%,1.000000,508.250000,193.850000,154.000000,12.000000,3.967500
50%,1.000000,2077.000000,538.050000,482.370000,45.220000,10.730000
75%,3.000000,6961.750000,2633.565000,2217.292500,221.042500,26.377500
max,3226.000000,120668.000000,17332.900000,16618.450000,14000.000000,10311.500000


In [53]:
print(f"${above_max.monthly_inventory_sales.sum():,.2f}")

$14,237,507.14


In [54]:
ttm_above_max

item_type  manufacturer  \
month   sku                                     
2024-04 10891    Inventory Item         Watts   
        1165799  Inventory Item  Consolidated   
        12629    Inventory Item        Crosby   
        1333     Inventory Item        Kunkle   
        141      Inventory Item  Consolidated   
...                         ...           ...   
2025-04 2004826  Inventory Item          Asco   
        617998   Inventory Item  Consolidated   
        6701     Inventory Item  Consolidated   
        7236     Inventory Item  Consolidated   
        804104   Inventory Item      Fastenal   

                                                   item_name  \
month   sku                                                    
2024-04 10891    Inactivated + 174A .75 50# WATTS HW SECT IV   
        1165799                                      1165799   
        12629                                         200289   
        1333     6021EDT01A .75 KUNKLE SV  (Steam Use ONLY!)   
        141                                .75 2478D-1-31-DA   
...                                                      ...   
2025-04 2004826                                   EF8342G001   
        617998                                        617998   
        6701                  4347701 DISC HOLDER-MONEL 1720   
        7236                 .75 19096LC-2-CC-MS-31-FT-FT-GS   
        804104                                        804104   

                                                       description   qty  \
month   sku                                                                
2024-04 10891       174A .\r\n75" WATTS SV\r\nSET @ 50 PSI HW {IV}   1.0   
        1165799                         LG434\r\nLEVER GROUP PLAIN   2.0   
        12629                             200289 Crosby Gasket Kit   3.0   
        1333     6021EDT01A \r\n.75" KUNKLE SV\r\n(TEFLON SEAT ...  15.0   
        141                 2478D-1-31-DA \r\n.75" CONSOLIDATED SV   7.0   
...                                                            ...   ...   
2025-04 2004826  EF8342G001 120/60 ASCO SOLENOID VALVE, EXPLOSI...   6.0   
        617998                          LTR09 \r\nLiquid Trim Kit    1.0   
        6701                        4347701 DISC HOLDER-MONEL 1720   1.0   
        7236              19096LC-2-FT-FT\r\n.75" CONSOLIDATED SRV   1.0   
        804104   IK10-150 B7 Stud Bolt Kit – (12) 7/8”-9 X 4-3/...   3.0   

                   sales  quoted_cost  max_cost  difference  pct_diff  
month   sku                                                            
2024-04 10891     462.00       323.40   223.200     100.200     44.89  
        1165799  1184.00       552.09   504.000      48.090      9.54  
        12629    2411.00       599.33   587.000      12.330      2.10  
        1333     3840.00       150.42   145.600       4.820      3.31  
        141      2277.00       188.58   179.340       9.240      5.15  
...                  ...          ...       ...         ...       ...  
2025-04 2004826  3138.00       392.49   334.865      57.625     17.21  
        617998   1618.42      1392.00   678.530     713.470    105.15  
        6701     5461.00      3907.13  3418.740     488.390     14.29  
        7236     1973.00      1301.92   580.060     721.860    124.45  
        804104    300.00        59.65    43.050      16.600     38.56  

[422 rows x 10 columns]